In [ ]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'
BASE_CONFIGURATION_NAME = 'configuration'


In [ ]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database
import matplotlib.pyplot as plt


In [ ]:
AGGREGATE_NUMBER = 20 #the number of neighbor points to sum to plot the needed graphs

# Load the experiment

In [ ]:
if True:

    base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-dqn\\HPOpt\\first\\experiments"
    
    experiment_path = f'{base_experiment_path}'

In [ ]:
if not os.path.exists(experiment_path):
    raise Exception("DOES NOT EXIST")

In [ ]:
hp_experiment_paths = [
            os.path.join(experiment_path, f)
            for f in os.listdir(experiment_path)
        ]

hp_experiment_names = [os.path.basename(hp_experiment_path) for hp_experiment_path in hp_experiment_paths]

# Evaluation of HyperparameterOptimizationPipeline

In [ ]:
hp_optuna_studies = {}

for i in range(len(hp_experiment_paths)):

    hp_optuna_studies[hp_experiment_names[i]] = load_study_from_database(database_path=hp_experiment_paths[i] + '\\' + OPTUNA_DATABASE)

## Hyperparameter Study

In [ ]:
for key, optuna_study in hp_optuna_studies.items():

    print(f"For HP optimization {key}:")

    try:
        print(f"    optuna_study done with with best value {optuna_study.best_value} in trial {optuna_study.best_trial.number} with best parameters:\n{optuna_study.best_params}")

    except:
        print("    No best trial yet")

    print()

In [ ]:
from optuna.importance import get_param_importances

all_importances = {}


for key, optuna_study in hp_optuna_studies.items():
    print(f"For HP optimization {key}:")

    try:

        # compute parameter importances
        importances = get_param_importances(optuna_study)
        all_importances[key] = importances

        if len(importances) == 0:
            print("    No importances could be computed (maybe too few completed trials).")
        else:
            print("    Parameter importances:")
            for param, val in importances.items():
                print(f"        {param}: {val:.3f}")

    except Exception as e:
        print(f"    Could not compute results: {e}")

    print()

In [ ]:
CHARS_MAX = 40

# --- comparison ---
if all_importances:
    print("=== Cross-study comparison of parameter importances ===")
    # collect all parameter names
    all_params = set()
    for imp in all_importances.values():
        all_params.update(imp.keys())

    # print a table-like comparison
    header = "Parameter".ljust(CHARS_MAX) + " " +  " ".join([name[:CHARS_MAX - 1].ljust(CHARS_MAX - 1) for name in all_importances.keys()])
    print(header)
    print("-" * len(header))
    for param in all_params:
        row = param[:CHARS_MAX].ljust(CHARS_MAX)
        for study_name in all_importances.keys():
            val = all_importances[study_name].get(param, 0.0)
            row += f" {val:.3f} ".ljust(CHARS_MAX)
        print(row)

In [ ]:
from optuna.importance import get_param_importances

all_importances = {}


for key, optuna_study in hp_optuna_studies.items():
    print(f"For HP optimization {key}:")

    

    fig = vis.plot_param_importances(optuna_study)
    fig.show()

In [ ]:


for key, optuna_study in hp_optuna_studies.items():
    print(f"For HP optimization {key}:")



    fig = vis.plot_parallel_coordinate(optuna_study)
    fig.show()

In [ ]:


for key, optuna_study in hp_optuna_studies.items():
    print(f"For HP optimization {key}:")



    fig = vis.plot_intermediate_values(optuna_study)
    fig.show()

In [ ]:
    

for key, optuna_study in hp_optuna_studies.items():
    print(f"For HP optimization {key}:")



    fig = vis.plot_optimization_history(optuna_study)
    fig.show()